# **Laboratory 0 - Introduction to Jupiter Notebook**
In this notebook, you will interact with the Jupyter Notebook environment to verify that you can run the code correctly without any issues. You will have to answer a couple of trivial questions *unique* for each student.
It is the opportunity to solve potential problems before the first laboratory.

## **Instructions**

**Deliverable.**
You should complete this notebook and submit a `.zip` file containing this notebook `.ipynb` file.

**Deadline.**
The submission deadline is on the 21st February 2025 at 23:59.

**Submission.**
Submit your notebook on [Gradescope](https://www.gradescope.com/) by logging in with your account `@student.uliege.be`.
If the course does not appear in your dashboard on Gradescope, contact us on [Ed Discussion](https://edstem.org/us/dashboard) quickly. If you are not familiar with submitting your work to Gradescope, you will find all the necessary information in the [online help](https://help.gradescope.com/article/ccbpppziu9-student-submit-work).

**Collaboration policy.**
You can discuss the assignment with other students, but *you must write your own solutions, and write and run your own code*. Copying someone else's solution, or just making trivial changes to avoid copying verbatim, is not acceptable.

## **Notebook setup**
You must run this piece of code and enter your student email address before running anything else.
The email address will be used to generate your unique questions so make sure it is the exact same one as the one you are using on [Gradescope](https://www.gradescope.com/).

In [ ]:
# @title
from ipywidgets import Dropdown, Layout, Button, Box, \
    HBox, VBox, Label, Text, HTML, BoundedIntText, HTMLMath, interactive_output, \
    Output, IntSlider, Valid
from IPython.display import display, Math, Latex, Markdown
import re
import hashlib
import numpy as np
from torchvision.datasets import CIFAR10, Imagenette
import matplotlib.pyplot as plt
# import logging

# Test other packages needed for the other laboratories
try :
    import control
except:
    print("/!\ python-control package is not installed" )
    !pip install control
    import control

try :
    import mujoco
except:
    print("/!\ mujoco-py package is not installed" )
    !pip install mujoco
    import mujoco

try :
    import scipy
except:
    print("/!\ scipy package is not installed" )
    !pip install scipy
    import scipy


In [ ]:
# @title
# Styles and classes
display(HTML("<style>.red_label { color:red; font-weight: bold }</style>"),
        HTML("<style>.green_label { color:green; font-weight: bold }</style>"))

class Validation():
    def __init__(self, check_method, answer):
        self._check_method = check_method
        self._widget = Label(value = '')
        self.valid = Valid(value=None)
        self.answer = answer

    def __call__(self, arg):
        flag = self._check_method(arg)
        flag = int(flag)
        color = ['green_label','red_label']
        valid = 'OK' if flag else 'Try again...'
        self._widget.value = valid
        self._widget.add_class(color[1-flag])
        self._widget.remove_class(color[flag])
        self.valid.value = flag

    @property
    def widget(self):
        return self._widget

    @property
    def value(self):
        return self._widget.value

class Quiz():
    def __init__(self, options, colors, answer):
        self.answer = answer
        self.user_answer = Label(value='')

        self.validation = Validation(self._check_meth, self.answer)

        self.button_layout = Layout(display='flex', height='auto',
                                    width='40%', min_height='100px',
                                    align_items='center', justify_content='center')
        self.buttons = [Button(description=option,
                               layout=self.button_layout) for option in options]
        self.stylize_buttons(self.buttons, colors)

        self.grid_lay = Layout(display='flex', justify_content='center',
                               align_items='center', width='100%')
        self.wrapped_buttons = self._box_buttons()

    def stylize_buttons(self, buttons, colors):
        for button, color in zip(buttons, colors):
            button.style.button_color = color
            button.style.font_weight = 'bold'
            button.style.font_size = button.layout.min_height
            button.on_click(self.validation)

    def _box_buttons(self):
        tmp = [HBox([self.buttons[i], self.buttons[i+1]],
                    layout=self.grid_lay) for i in range(0, len(self.buttons), 2)]
        tmp.append(self.validation.widget)
        wrap_buttons = VBox(tmp, layout=self.grid_lay)
        return wrap_buttons

    def _check_meth(self, arg):
        self.user_answer.value = arg.description
        return arg.description == self.answer

    def get_user_answer(self):
        return self.validation.user_answer

    def display(self):
        display(self.wrapped_buttons)

    def _get_answer(self):
        return self.answer

class Addition():
    def __init__(self, numbers):
        self.numbers = numbers
        self.user_answer = Label(value='')
        self.user_answer.layout.visibility = 'hidden'

        self.validation = Validation(self._check_meth, sum(self.numbers))
        self.slider = BoundedIntText(
                                value=None,
                                min=-100,
                                max=100,
                                step=1,
                                description='Answer:',
                                disabled=False)
        self.slider.observe(self.validation, names='value')
        self.grid_lay = Layout(display='flex', justify_content='center',
                               align_items='center', width='100%', height='auto')
        self.wrapped = self._box_elements()

    def _disp_question(self):
        text = f'{self.numbers[0]} + {self.numbers[1]} = ?'
        label = Label(value=text)
        label.style.font_weight = 'bold'
        label.style.font_size = '50px'
        # label.style.margin = '5px'
        # label.style.text_align = 'center'
        label.layout.align_items = 'center'
        label.layout.min_height = '100px'
        label.layout.display = 'flex'

        return label

    def _box_elements(self):
        tmp = HBox([self.slider, self.validation.widget], layout=self.grid_lay)
        wrap_elements = VBox([self._disp_question(), tmp], layout=self.grid_lay)
        return wrap_elements


    def _check_meth(self, arg):
        self.user_answer.value = str(arg['new'])
        return arg['new'] == sum(self.numbers)

    def display(self):
        display(self.wrapped)

    def _get_answer(self):
        return sum(self.numbers)

class ImageQuestion():
    def __init__(self, img, answer, choices):
        self.img = img
        self.answer = answer
        self.user_answer = Label(value='')

        self.validation = Validation(self._check_meth, self.answer)
        self.dropdown = Dropdown(
                    options=choices,
                    value=None,
                    description='Choose:',
                    display='flex',
                    justify_content='flex-end',
                    width='150%',)

        self.dropdown.observe(self.validation, names='value')
        self.grid_lay = Layout(display='flex', justify_content='center',
                               align_items='center', width='100%')
        self.img_out = Output()
        self.wrapped = self._box_elements()

    def _disp_question(self):
        with self.img_out:
            plt.figure(figsize=(4, 4), dpi=100);
            plt.imshow(self.img, interpolation='sinc', aspect='equal')
            plt.axis('off');
            plt.show();

        return self.img_out

    def _box_elements(self):
        img_quest = self._disp_question()
        wrap_elements = HBox([img_quest, self.dropdown, self.validation.widget], layout=self.grid_lay)
        return wrap_elements

    def _check_meth(self, arg):
        self.user_answer.value = str(arg['new'])
        return arg['new'] == self.answer

    def display(self):
        display(self.wrapped)

    def _get_answer(self):
        return self.answer

class ShapeQuestion():
    def __init__(self, answer):
        self.answer = answer
        self.user_answer = Label(value='')

        self.validation = Validation(self._check_meth, self.answer)
        self.shape_slider = IntSlider(min=3, max=10, step=1, value=3)
        self.mpl_out = interactive_output(self.plot_shape, {'nb_edges' : self.shape_slider})

        self.shape_slider.observe(self.validation, names='value')
        self.grid_lay = Layout(display='flex', justify_content='center',
                               align_items='center', width='100%')
        self.wrapped = self._box_elements()

    def plot_shape(self, nb_edges):
        plt.figure(figsize=(4, 4), dpi=100);
        x = np.cos(np.linspace(-3*np.pi/2, np.pi/2, nb_edges+1))
        y = np.sin(np.linspace(-3*np.pi/2, np.pi/2, nb_edges+1))
        plt.plot(x, y, linewidth=4)
        plt.axis('off')
        plt.show()

    def _box_elements(self):
        tmp = VBox([self.shape_slider, self.validation.widget], layout=self.grid_lay)
        wrap_elements = VBox([self.mpl_out, tmp], layout=self.grid_lay)
        return wrap_elements

    def _check_meth(self, arg):
        self.user_answer.value = str(arg['new'])
        return arg['new'] == self.answer

    def display(self):
        display(self.wrapped)

    def _get_answer(self):
        return self.answer

class Summary():
    def __init__(self, questions):
        self.names, self.questions = zip(*questions)
        self.names = list(self.names)
        self.questions = list(self.questions)
        self.validations = [question.validation.valid for question in self.questions]
        self.output = interactive_output(self._disp_summary, dict(zip(self.names, self.validations)))
        self.metadata = Label(value=str(self.get_metadata()))
        self.set_observers()

    def get_metadata(self):
        metadata = {}
        for name, question in zip(self.names, self.questions):
            metadata[name] = question.user_answer.value
        return metadata

    def set_observers(self):
        for question in self.questions:
            question.user_answer.observe(self._on_change, names='value')

    def _on_change(self, change):
        self.metadata.value = str(self.get_metadata())
        print('', end='\r', flush=True)
        print(self.metadata.value, end='')

    def _disp_summary(self, **kwargs):
        args = list(kwargs.values())
        question_list = []
        correct = 0
        total_q = len(args)
        head_style = 'span style="font-size: 2vw"'
        for name, val in zip(self.names, args):
            if val:
                q_summary = HTML(value=f'<{head_style}> {name}: <span style="color:green; font-weight:bold">Correct</span></{head_style}>')
                correct += 1
            else:
                q_summary = HTML(value=f'<{head_style}> {name}: <span style="color:red; font-weight:bold">Incorrect</span></{head_style}>')
            question_list.append(q_summary)
        score_summary = HTML(value=f'<{head_style}><span style="font-weight:bold">Score</span>: {correct}/{total_q} </{head_style}>')
        quest_summary = VBox(question_list + [score_summary],
                             layout=Layout(display='flex', align_items='flex-start',
                                           justify_content='center', width='30%'))
        if correct == total_q:
            congrats = HTML(value=f'<span style="color:green; font-weight:bold; font-size:3vw">🎉 Congratulations,<br><br>Quiz is done! 🎉</span>',
                            layout=Layout(display='flex', justify_content='flex-end', width='30%'))
            quest_summary = HBox([quest_summary, congrats], layout=Layout(display='flex', align_items='center',
                                                                          justify_content='space-around', width='100%'))

        display(quest_summary)

    def display(self):
        display(self.output)

display(Markdown('---\n# Enter your email address:'))
name = Text(description='', placeholder='Enter your email')
valid_mail = Valid(value=None)
d_box = HBox([name, valid_mail], layout=Layout(text_color='blue',
                                               justify_content='flex-start',
                                               width='100%'))

def validate_email(email):
    pattern = r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$'
    return re.match(pattern, email) is not None

def on_name_change(change):
    valid_mail.value = validate_email(change['new'])

name.observe(on_name_change, names='value')
display(d_box)
display(Markdown('---\n'))


## **Personalized quiz**
Run the following cell **after** you have entered your student email address. It will generate a personalized quiz for you.
It should take you **less than 5 minutes** to complete it.
For each question, you will have immediate feedback on whether you answered correctly or not.

**Make sure you answer all the questions correctly <span style="color:red">before</span> submitting the notebook.**
To help you, a summary of your answers will be displayed at the end of the quiz.

In [ ]:
# @title
if not valid_mail.value:
    display(Markdown('## <span style="color:red">⚠️ **Please enter a valid email address** ⚠️</span>'))
else:
    hash = hashlib.sha256(name.value.lower().encode())
    seed = np.frombuffer(hash.digest(), dtype='uint32')
    rstate = np.random.RandomState(seed)

    # Computation answer
    numbers = rstate.randint(-50, 50, (2, 2))
    addition_quiz = [Addition(nb) for nb in numbers]

    # Image question
    try:
      imagenette = Imagenette(root='.', split='val', download=True, size='160px')
    except:
      imagenette = Imagenette(root='.', split='val', download=False, size='160px')
    img_idx = rstate.randint(0, len(imagenette))
    img, cat = imagenette[img_idx]
    img_options = [('Tench', 0),
                ('English springer', 1),
                ('Cassette player', 2),
                ('Chain saw', 3),
                ('Church', 4),
                ('French horn', 5),
                ('Garbage truck', 6),
                ('Gas pump', 7),
                ('Golf ball', 8),
                ('Parachute', 9)]
    img_quiz = ImageQuestion(img, cat, img_options)

    # Shape question
    shape = rstate.randint(3, 10)
    shape_quiz = ShapeQuestion(shape)

    # Quiz
    quiz_options = ['A', 'B', 'C', 'D']
    quiz_colors = ['#3498db', '#27ae60', '#e74c3c', '#f1c40f']
    rstate.shuffle(quiz_colors)
    quiz_answer = rstate.choice(quiz_options)
    quiz = Quiz(quiz_options, quiz_colors, quiz_answer)

    # Summary of the quiz
    summary = Summary([('Question 1', addition_quiz[0]),
                       ('Question 2', img_quiz),
                       ('Question 3', shape_quiz),
                       ('Question 4', quiz),
                       ('Question 5', addition_quiz[1])])

    # Display Quiz and questions in Markdown
    display(Markdown('---\n## **Question 1 :** Addition\n' + \
                     f'### Type in the result of the addition displayed.'))
    addition_quiz[0].display()

    display(Markdown('---\n## **Question 2 :** Image Recognition\n' + \
                     f'### Choose from the dropdown menu the correct label to associate with the image shown.'))
    img_quiz.display()

    display(Markdown('---\n## **Question 3 :** Select the Shape\n' + \
                     f'### Use the slider to change the number of edges of the shape to have {shape} edges.'))
    shape_quiz.display()

    color_idx = quiz_options.index(quiz_answer)
    display(Markdown('---\n## **Question 4 :** Pick the Color\n' + \
                     f'### Click on the button with letter **{quiz_answer}**.'))
    quiz.display()

    display(Markdown('---\n## **Question 5 :** Addition, Again !\n' + \
                     f'### Type in the result of the addition displayed.'))
    addition_quiz[1].display()

    display(Markdown('---\n# **Summary**'))
    summary.display()

